In [ ]:
from cfg.configfile_req import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

    

In [ ]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

In [ ]:
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_rq_tracker_id)
my_project_issues = sorted(tmp, key=lambda k: k.custom_fields.get(req_chapter_cf_id).value)
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_doc_tracker_id)
my_doc_issues = sorted(tmp, key=lambda k: k.custom_fields.get(req_chapter_cf_id).value)

Ahora recorremos el proyecto y sacamos los diagramas completos de jerarquía y dependencias, y guardamos los ficheros de esos diagramas en la carpeta doc.

In [ ]:

# Preparamos el fichero JSON que usaremos de puente para el resultado

result_list = []

def parent_child_validation(i,results):
    # Exploro los hijos
    for c in i.children:
        child_issue = redmine.issue.get(c.id)
        if (req_status_maturity[child_issue.status.name] < req_status_maturity[i.status.name]):
            print("\n\n**************")
            print(i.id,": ",i.subject,": ",i.status,":",req_status_maturity[i.status.name])
            print("\t* ",child_issue.id,": ",child_issue.subject,": ",child_issue.status,":",req_status_maturity[child_issue.status.name])
            print("xxxxxxxxxxxx: Error.  el requisito hijo está en estado ",child_issue.status," mientras su requisito padre está en estado ",i.status)
            thiserror = {
                'type':'hyerarchy',
                'parent':{
                    'id': i.id,
                    'subject': i.subject,
                    'status': i.status.name,
                    'status_maturity': req_status_maturity[i.status.name],
                },
                'child':{
                    'id': child_issue.id,
                    'subject': child_issue.subject,
                    'status': child_issue.status.name,
                    'status_maturity': req_status_maturity[child_issue.status.name],
                },
            }
            results.append(thiserror)



def dependence_validation(i,results):            
    # Exploro las relaciones
    my_issue_relations = redmine.issue_relation.filter(issue_id=i.id)
    my_filtered_issue_relations = list(filter(lambda x: x.issue_to_id != i.id, my_issue_relations))
    if (len(my_issue_relations)>0):
        for r in my_filtered_issue_relations:
            rel_issue = redmine.issue.get(r.issue_to_id)
            if (req_status_maturity[rel_issue.status.name] > req_status_maturity[i.status.name]):
                print("\n\n**************")
                print(i.id,": ",i.subject,": ",i.status,":",req_status_maturity[i.status.name])
                print("\t-",r.relation_type,"-> ",rel_issue.subject," : ",rel_issue.status,":",req_status_maturity[rel_issue.status.name])
                print("xxxxxxxxxxxx: Error.  el requisito dependiente está en estado ",rel_issue.status," mientras el requisito del que depende está en estado ",i.status)
                thiserror = {
                    'type':'dependency',
                    'dependable':{
                        'id': i.id,
                        'subject': i.subject,
                        'status': i.status.name,
                        'status_maturity': req_status_maturity[i.status.name],
                    },
                    'dependent':{
                        'id': rel_issue.id,
                        'subject': rel_issue.subject,
                        'status': rel_issue.status.name,
                        'status_maturity': req_status_maturity[rel_issue.status.name],
                    },
                }
                results.append(thiserror)

for issue in my_project_issues:
    dependence_validation(issue,result_list)
    parent_child_validation(issue,result_list)

print("Acabamos")

In [ ]:
import json

# Preparamos el fichero JSON que usaremos de puente para el resultado
  
with open('./RqValidation-Result.json', 'w') as outfile:  
    json.dump(result_list, outfile)

In [ ]:
req_maturity_propagation = ['RqZombie','RqDraft','RqStable','RqApproved']

def propagate_maturity(rq,mat_st,mat_lev):
    # Actualizamos el estado del requisito
    print("rq:",rq)
    print("this_rq_st: ",rq.status.name)
    this_rq_mat_lev = req_status_maturity[rq.status.name]
    print("this_rq_mat_lev: ",this_rq_mat_lev)
    print("mat_lev:", mat_lev)
    if (mat_lev<this_rq_mat_lev):
        # Nos toca propagar el cambio
        redmine.issue.update(resource_id=rq.id,
                             status_id = mat_st)        
        # Propagaremos el cambio a aquellas relaciones dependientes de madurez superior
        rel = redmine.issue_relation.filter(issue_id=rq.id)
        rel_filt = list(filter(lambda x: x.issue_to_id != rq.id, rel))
        if (len(rel_filt)>0):
            for r in rel_filt:
                req_to_change = redmine.issue.get(r.issue_to_id)
                propagate_maturity(req_to_change,mat_st,mat_lev)

    

for e in result_list:
    if (e['type'] == 'dependency'):
        mat = (e['dependable'])['status_maturity']
        dependent_id = e['dependent']['id']
        print(dependent_id)
        maturity_str = req_maturity_propagation[mat]
        st_list = redmine.issue_status.all()
        status_to_set = None
        for st in st_list:
            if (st.name == maturity_str):
                print(st.name)
                status_to_set = st
                
        if status_to_set is not None:
            print("Propagamos: ",dependent_id, " ", status_to_set.id)
            req_to_change = redmine.issue.get(dependent_id)
            propagate_maturity(req_to_change,status_to_set.id, mat)
            print("Acabamos con ",e)
        else:
            print("ERROR, no pudimos encontrar el estado")
        

print("Acabamos!!!")